In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/yhu5/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
from glob import glob
from itertools import groupby, combinations
from sklearn.metrics import cohen_kappa_score, precision_score, recall_score, precision_recall_fscore_support,confusion_matrix
import numpy as np
from nltk.corpus import stopwords
stops = stopwords.words('english')

PHASES = ('starting_spans', 'hierarchical_labels')
LABEL_DECODERS = { \
  PHASES[0] : { \
      'participants':  { 0: 'No Label', 1: 'p' },
      'interventions': { 0: 'No Label', 1: 'i' },
      'outcomes':      { 0: 'No Label', 1: 'o' }
    },
  PHASES[1]: { \
      'participants': { \
        0: 'No label',
        1: 'Age',
        2: 'Sex',
        3: 'Sample-size',
        4: 'Condition' },

      'interventions': { \
        0: 'No label',
        1: 'Surgical',
        2: 'Physical',
        3: 'Pharmacological',
        4: 'Educational',
        5: 'Psychological',
        6: 'Other',
        7: 'Control' },

      'outcomes': { \
        0: 'No label',
        1: 'Physical',
        2: 'Pain',
        3: 'Mortality',
        4: 'Adverse-effects',
        5: 'Mental',
        6: 'Other' }
    }
}

EBM_NLP = ''

def fname_to_pmid(f):
  return f.split('/')[-1].split('.')[0]
def apply_bitmask(arr, mask):
  return [x for x, m in zip(arr, mask) if m]
def condense_labels(labels):
  groups = [(k, sum(1 for _ in g)) for k,g in groupby(labels)]
  spans = []
  i = 0
  for label, length in groups:
    if label != 0:
      spans.append((label, i, i+length))
    i += length
  return spans

def get_test_labels(ebm_nlp_dir, phase, pio):
  test_dir = '%s/annotations/aggregated/%s/%s/test/gold/' %(EBM_NLP, phase, pio)
  test_fnames = glob('%s/*.ann' %test_dir)
  print('Loading %d anns from %s' %(len(test_fnames), test_dir))
  return { fname_to_pmid(fname): open(fname).read().split() for fname in test_fnames }

#def span_overlap(pmids, pred_labels, test_labels, labels):
#  for pmid in pmids:
#    test_spans = condense_labels(test_labels[pmid])
#    pred_spans = condense_labels(pred_labels[pmid])
#    for tspan in test_spans:
#      for pspan in pred_spans:
#        pass
  
def vanilla_tokens(pmids, pred_labels, test_labels, labels):
  y_pred = []
  y_test = []
  for pmid in pmids:
    assert len(pred_labels[pmid]) == len(test_labels[pmid])
    y_pred += pred_labels[pmid]
    y_test += test_labels[pmid]
  token_f1(true = y_test, pred = y_pred, labels = labels)

def sw_tokens(pmids, pred_labels, test_labels, labels):
  y_pred = []
  y_test = []
  for pmid in pmids:
    assert len(pred_labels[pmid]) == len(test_labels[pmid])
    tokens = open('%s/documents/%s.tokens' %(EBM_NLP, pmid)).read().split()
    token_mask = [t in stops for t in tokens]
    y_pred += apply_bitmask(pred_labels[pmid], token_mask)
    y_test += apply_bitmask(test_labels[pmid], token_mask)
  token_f1(true = y_test, pred = y_pred, labels = labels)

def eval_labels(ebm_nlp_dir, pred_labels, phase, pio, eval_func = vanilla_tokens):
  global EBM_NLP
  EBM_NLP = ebm_nlp_dir

  print('Evaluating labels for %s %s' %(phase, pio))
  test_labels = get_test_labels(EBM_NLP, phase, pio)
  pmids = set(test_labels.keys()) & set(pred_labels.keys())
  print('Checking labels for %d pmids (out of %d possible test docs)' %(len(pmids), len(test_labels)))

  labels = set(LABEL_DECODERS[phase][pio].keys())
  labels.remove(0)
  labels = [str(l) for l in labels]

  eval_func(pmids, pred_labels, test_labels, labels)

def get_f1(prec, rec):
  return 2*prec*rec/(prec+rec)

def token_f1(true, pred, labels):

  class_scores = zip(labels, precision_score(true,pred, labels = labels,average=None), recall_score(true,pred, labels = labels,average=None))
  for label, prec, rec in class_scores:
    print ('{label}\t{f1:.3f}\t{prec:.3f}\t{rec:.3f}'.format(label=label,f1=get_f1(prec, rec),prec=prec,rec=rec))
    
  prec = precision_score(true, pred, labels = labels, average='micro')
  rec = recall_score(true, pred, labels = labels, average='micro')
  f1 = get_f1(prec, rec)
  print ('overall\t{f1:.3f}\t{prec:.3f}\t{rec:.3f}'.format(f1=f1,prec=prec,rec=rec))
    



In [40]:
gold = []
pre = []
with open('/home/yhu5/jianfu_NER/ner_ft_bert/data/EBM_raw_starting_level/output_1e-5_5ep/fold1/output_update_256/microsoft-BiomedNLP-PubMedBERT-base-uncased-abstract/predict_test_label.txt','r') as f:
    for line in f:
        if not line.startswith('-DOCSTART-') and not line=='\n':
            gold.append(line.split()[2])
            pre.append(line.split()[1])
token_f1(true = gold, pred = pre, labels = ['I-i', 'B-p', 'I-p', 'B-o', 'I-o', 'B-i'])

/data/yhu5/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/yhu5/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_39078/680363953.py:110: RuntimeWarning: invalid value encountered in double_scalars
  return 2*prec*rec/(prec+rec)


I-i	0.652	0.750	0.576
B-p	nan	0.000	0.000
I-p	0.827	0.914	0.755
B-o	nan	0.000	0.000
I-o	0.709	0.614	0.838
B-i	nan	0.000	0.000
overall	0.732	0.730	0.734


In [46]:
for i in range(5):
    gold = []
    pre = []
    with open('/home/yhu5/jianfu_NER/ner_ft_bert/data/pure_AD_5fold_section_filtered/output_major_revision_check/fold{}/output_update_256/microsoft-BiomedNLP-PubMedBERT-base-uncased-abstract/predict_test_label.txt'.format(i+1),'r') as f:
        for line in f:
            if not line.startswith('-DOCSTART-') and not line=='\n' and len(line.split())==3:
                gold.append(line.split()[2].replace('B-','I-'))
                pre.append(line.split()[1].replace('B-','I-'))
    token_f1(true = gold, pred = pre, labels = ['I-P','I-I','I-C',  'I-O'])
    print ('\n')

I-P	0.951	0.919	0.986
I-I	0.707	0.898	0.582
I-C	0.769	0.882	0.682
I-O	0.918	0.992	0.854
overall	0.908	0.958	0.862


I-P	0.959	0.988	0.931
I-I	0.919	0.919	0.919
I-C	0.818	1.000	0.692
I-O	0.916	0.879	0.956
overall	0.931	0.927	0.935


I-P	0.844	0.906	0.789
I-I	0.824	0.857	0.793
I-C	0.760	0.950	0.633
I-O	0.913	0.909	0.916
overall	0.862	0.897	0.829


I-P	0.969	0.943	0.996
I-I	0.853	0.844	0.863
I-C	0.828	0.706	1.000
I-O	0.878	0.866	0.890
overall	0.905	0.886	0.924


I-P	0.792	0.857	0.736
I-I	0.824	0.856	0.794
I-C	0.870	0.833	0.909
I-O	0.825	0.780	0.876
overall	0.811	0.837	0.785


